In [88]:
import pandas as pd
import datetime as dt
import numpy as np

In [45]:
# Python program to demonstrate
# command line arguments


import argparse


# Initialize parser
parser = argparse.ArgumentParser()

# Adding optional argument
parser.add_argument("-i", "--to_process", help = "to_process")
parser.add_argument("-o", "--output_dir", help = "output_dir")

# Read arguments from command line
args = parser.parse_args()




usage: ipykernel_launcher.py [-h] [-i TO_PROCESS] [-o OUTPUT_DIR]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\raazx\AppData\Roaming\jupyter\runtime\kernel-ed3c9108-2029-4fc7-8841-b5516e7cb8f4.json


SystemExit: 2

C:\Python\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
dff=pd.read_parquet('evaluation_data/input/raw_data.parquet', engine='auto')

In [ ]:
dff

In [ ]:
def trip(z):
    global trip_no
    if z>= pd.Timedelta('0 days 07:00:00'):
        trip_no += 1
        return trip_no
    else:
        return trip_no
    
    

def process_1(df,vehicle_code):
    df.timestamp = pd.to_datetime(df.timestamp)
    df['time_diff']= df.timestamp.diff().abs()
    df['trip']= df['time_diff'].apply(lambda x: trip(x))
    
    for j in df.trip.unique():
        filename= '/'+str(vehicle_code) +'_'+str(j) +'.csv'
        if args.output_dir:
            df[df.trip==j][dff.columns].to_csv(str(args.output_dir) + filename, index= False)
            
        else:
            df[df.trip==j][dff.columns].to_csv('evaluation_data/output/process_1'+filename, index= False)
        
    


In [ ]:
for i in dff.unit.unique():
    trip_no=0
    process_1(dff[dff.unit==i],i)
    
    
    

In [46]:
import os

all_files = os.listdir("evaluation_data/output/process1")    
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))

In [47]:
import requests

url = 'https://apis.tollguru.com/toll/v2/gps-tracks-csv-upload?mapProvider=osrm&vehicleType=5AxlesTruck'

headers = {'x-api-key': 'PrpGFmGqDMPGgQfD8N6HhB9ft2826ppg', 'Content-Type': 'text/csv'}

for i in csv_files:
    file_path = path +'/'+ i
    with open(file_path, 'rb') as file:
        response = requests.post(url, data=file, headers=headers)
        
    if args.ouput_dir:
        with open(args.ouput_dir + i[:-4] + '.json', 'w') as f:
            data = json.loads(response.content)
            json.dump(data, f)
    else:
        with open("evaluation_data/output/process2/" + i[:-4] + '.json', 'w') as f:
            data = json.loads(response.content)
            json.dump(data, f)
            
            

NameError: name 'path' is not defined

In [59]:
url = 'https://apis.tollguru.com/toll/v2/gps-tracks-csv-upload?mapProvider=osrm&vehicleType=5AxlesTruck'

headers = {'x-api-key': 'PrpGFmGqDMPGgQfD8N6HhB9ft2826ppg', 'Content-Type': 'text/csv'}


file_path = 'evaluation_data/output/process1' +'/'+ '2000_1.csv'
with open(file_path, 'rb') as file:
    response = requests.post(url, data=file, headers=headers)
data=json.loads(response.content)

In [60]:
data.keys()

dict_keys(['status', 'summary', 'route', 'meta'])

In [65]:
data['route']['tolls']

[{'type': 'ticketSystem1',
  'tagCost': 3.77,
  'tagPriCost': 3.77,
  'tagSecCost': 3.77,
  'cashCost': 4.05,
  'licensePlateCost': None,
  'prepaidCardCost': 3.77,
  'currency': 'USD',
  'tagPrimaryNames': ['E-ZPass NJ'],
  'prepaidCardNames': [],
  'tagPrimary': ['E-ZPass NJ'],
  'tagSecondary': ['E-ZPass',
   'E-ZPass DE',
   'Uni',
   'I-Pass',
   'E-ZPass Skyway',
   'E-ZPass IN',
   'RiverLink',
   'E-ZPass ME',
   'E-ZPass MD',
   'E-ZPass MA',
   'MnPass',
   'E-ZPass NH',
   'E-ZPass NY',
   'E-ZPass NC',
   'E-ZPass OH',
   'E-ZPass PA',
   'E-ZPass RI',
   'E-ZPass VA',
   'E-ZPass WV',
   'E-Zpass Flex'],
  'licensePlateNames': ['Tolls by Mail'],
  'discountCarDetails': 'Other discounts: 65+ Senior citizens receive a 10% discount on the off-peak rate as Senior Citizen Discount. Vehicles with fuel efficiency 45 MPG and meeting California Super Ultra Low Emission Vehicle (SULEV) standard receive a 10% discount on the off-peak rate as Green Pass Discount. Must have NJ E-ZPass.

In [98]:
def check_null(x):
    if x is None:
        return ''
    else:
        return x

folder_path = "evaluation_data/output/process2"
all_files = os.listdir("evaluation_data/output/process2")    
json_files = list(filter(lambda f: f.endswith('.json'), all_files))
df=pd.DataFrame()

for i in json_files:
    unit= i.split('.')[0].split('_')[0]
    trip_id= i.split('.')[0].split('_')[1]
    file_path = 'evaluation_data/output/process2' +'/'+ i
    f = open(file_path)

    data = json.load(f)
    f.close()
    if 'route' in data.keys():
        if 'tolls' in data['route'].keys():
            for j in data['route']['tolls']:
                toll_system_type = check_null(j['type'])
                tag_cost = check_null(j['tagCost'])
                cash_cost= check_null(j.get('cashCost'))
                license_plate_cost = check_null(j['licensePlateCost'])

                toll_loc_id_start = check_null(j['start']['id'])
                toll_loc_id_end = check_null(j['end']['id'])
                toll_loc_name_start = check_null(j['start']['name'])
                toll_loc_name_end = check_null(j['end']['name'])
                entry_time= check_null(j['start']['arrival']['time'])
                exit_time= check_null(j['end']['arrival']['time'])

                dictionary={
                    'unit':unit,
                    'trip_id': trip_id,
                    'toll_loc_id_start': toll_loc_id_start,
                    'toll_loc_id_end': toll_loc_id_end,
                    'toll_loc_name_start': toll_loc_name_start,
                    'toll_loc_name_end': toll_loc_name_end,
                    'toll_system_type': toll_system_type,
                    'entry_time': entry_time,
                    'exit_time': exit_time,
                    'tag_cost': tag_cost,
                    'cash_cost': cash_cost,
                    'license_plate_cost': license_plate_cost
                }
                df_dictionary = pd.DataFrame([dictionary])
                df = pd.concat([df, df_dictionary], ignore_index=True)

df      

,unit,trip_id,toll_loc_id_start,toll_loc_id_end,toll_loc_name_start,toll_loc_name_end,toll_system_type,entry_time,exit_time,tag_cost,cash_cost,license_plate_cost
0,1000,0,130002600,130000000,226 - Carlisle,30 - Warrendale,ticketSystem1,2023-12-05T23:14:58Z,2023-12-06T04:31:04Z,100.00,,201.8
1,1000,0,130002600,130000000,226 - Carlisle,30 - Warrendale,ticketSystem1,2023-12-05T23:14:58Z,2023-12-06T04:31:04Z,100.00,,201.8
2,1000,1,130001000,130006000,75 - New Stanton,353 - Neshaminy Falls,ticketSystem1,2023-12-08T01:31:00Z,2023-12-08T08:58:29Z,143.10,,288.8
3,1000,1,126001000,126001400,6 - PA Turnpike/Florence,7 - Bordentown/Trenton,ticketSystem1,2023-12-08T09:13:43Z,2023-12-08T09:22:58Z,7.97,9.25,
4,1000,1,126001600,126002400,7A - I-195/Trenton/Shore Points,10 - I-287/Metuchen/Edison Twsp,ticketSystem1,2023-12-08T09:40:30Z,2023-12-08T10:25:56Z,10.64,12.15,
5,1000,2,126003000,126003200,13 - I-278/Eliz/Goethals/Verrazano,13A - Newark Aprt/Elizabeth Seaport,ticketSystem1,2023-12-05T14:05:57Z,2023-12-05T14:10:21Z,3.03,3.65,
6,1000,2,126003201,126002800,13A - Newark Aprt/Elizabeth Seaport,12 - Carteret/Rahway,ticketSystem1,2023-12-05T14:20:31Z,2023-12-05T14:30:31Z,3.59,4.05,
7,1000,2,126002800,126003200,12 - Carteret/Rahway,13A - Newark Aprt/Elizabeth Seaport,ticketSystem1,2023-12-05T14:40:01Z,2023-12-05T14:49:51Z,3.59,4.05,
8,1000,2,126003201,126002800,13A - Newark Aprt/Elizabeth Seaport,12 - Carteret/Rahway,ticketSystem1,2023-12-05T15:00:10Z,2023-12-05T15:10:09Z,3.59,4.05,
9,1000,3,130003200,130000000,247 - Harrisburg East,30 - Warrendale,ticketSystem1,2023-12-07T00:18:26Z,2023-12-07T06:08:13Z,108.90,,220


In [97]:
type(df.iloc[3]['cash_cost'])

float

In [79]:
data['route']['tolls']

[{'type': 'ticketSystem1',
  'tagCost': 3.77,
  'tagPriCost': 3.77,
  'tagSecCost': 3.77,
  'cashCost': 4.05,
  'licensePlateCost': None,
  'prepaidCardCost': 3.77,
  'currency': 'USD',
  'tagPrimaryNames': ['E-ZPass NJ'],
  'prepaidCardNames': [],
  'tagPrimary': ['E-ZPass NJ'],
  'tagSecondary': ['E-ZPass',
   'E-ZPass DE',
   'Uni',
   'I-Pass',
   'E-ZPass Skyway',
   'E-ZPass IN',
   'RiverLink',
   'E-ZPass ME',
   'E-ZPass MD',
   'E-ZPass MA',
   'MnPass',
   'E-ZPass NH',
   'E-ZPass NY',
   'E-ZPass NC',
   'E-ZPass OH',
   'E-ZPass PA',
   'E-ZPass RI',
   'E-ZPass VA',
   'E-ZPass WV',
   'E-Zpass Flex'],
  'licensePlateNames': ['Tolls by Mail'],
  'discountCarDetails': 'Other discounts: 65+ Senior citizens receive a 10% discount on the off-peak rate as Senior Citizen Discount. Vehicles with fuel efficiency 45 MPG and meeting California Super Ultra Low Emission Vehicle (SULEV) standard receive a 10% discount on the off-peak rate as Green Pass Discount. Must have NJ E-ZPass.

In [77]:
file_path = 'evaluation_data/output/process2' +'/'+ '2000_1.json'
f = open(file_path)
 
data = json.load(f)
f.close()

In [78]:
data

{'status': 'OK',
 'summary': {'countries': ['USA'],
  'currency': 'USD',
  'vehicleType': '5AxlesTruck',
  'vehicleDescription': 'Truck - 5 Axles',
  'fuelPrice': {'value': 3.65,
   'currency': 'USD',
   'units': 'USD/gallon',
   'fuelUnit': 'gallon'},
  'fuelEfficiency': {'city': 6.4,
   'hwy': 8.2,
   'units': 'MPG',
   'fuelUnit': 'gallon'},
  'units': {'currencyUnit': 'USD',
   'fuelEfficiencyUnit': 'MPG',
   'fuelUnit': 'gallon'},
  'departure_time': '2023-12-10T08:19:23Z',
  'departureTime': '2023-12-10T08:19:23Z',
  'share': {'name': 'Tollguru GPS Tracks',
   'prefix': 'Tollguru+GPS+Tracks',
   'uuid': '0d6c180d-82cb-4336-9fb9-2b5fff65f810',
   'timestamp': '2023-12-09T16:37:34.092480349Z',
   'client': 'GPSTracksAPI'}},
 'route': {'hasTolls': True,
  'distance': {'text': '83 mi', 'metric': '133 km', 'value': 133683},
  'costs': {'fuel': 36.47,
   'tag': 18.16,
   'cash': None,
   'licensePlate': None,
   'prepaidCard': 18.16,
   'tagAndCash': 18.16,
   'minimumTollCost': 18.16}